The main purpose of this notebook is to explore the dataset


In [1]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# Load the dataset


# Define the base directory containing data files
base_dir = "/Users/mjh/code/Obispodino/cvino/raw_data/last"  # Updated to absolute path
wines_file = os.path.join(base_dir, "XWines_Full_100K_wines.csv")
wines_data = pd.read_csv(wines_file)
wd = wines_data
wd.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10003,Monte Paschoal,http://www.montepaschoal.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."


In [3]:
# !pip install geopy

In [4]:
import pandas as pd
import numpy as np
import pickle
import os
from pathlib import Path
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

def geocode_regions(df, region_column='RegionName', cache_file='geocoding_cache.pkl', min_delay=1.0):
    """
    Geocode region names with caching and rate limiting

    Parameters:
        df (pd.DataFrame): Input DataFrame
        region_column (str): Name of region column
        cache_file (str): Path to cache file
        min_delay (float): Minimum delay between API requests (seconds)

    Returns:
        pd.DataFrame: DataFrame with latitude/longitude columns
    """
    # Initialize geocoder with rate limiter
    geolocator = Nominatim(user_agent="regional_analysis_app")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=min_delay)

    # Load existing cache
    cache_path = Path(cache_file)
    if cache_path.exists():
        with open(cache_path, 'rb') as f:
            geocode_cache = pickle.load(f)
    else:
        geocode_cache = {}

    # Get unique regions needing geocoding
    unique_regions = df[region_column].dropna().unique()
    new_regions = [r for r in unique_regions if r not in geocode_cache]

    print(f"Found {len(unique_regions)} unique regions ({len(new_regions)} new)")

    # Geocode new regions with retry logic
    for region in new_regions:
        for attempt in range(3):  # 3 retry attempts
            try:
                location = geocode(region)
                geocode_cache[region] = {
                    'latitude': location.latitude if location else np.nan,
                    'longitude': location.longitude if location else np.nan
                }
                break
            except Exception as e:
                if attempt == 2:  # Final attempt failed
                    geocode_cache[region] = {'latitude': np.nan, 'longitude': np.nan}
                    print(f"Failed to geocode {region} after 3 attempts")
                continue

    # Save updated cache
    with open(cache_path, 'wb') as f:
        pickle.dump(geocode_cache, f)

    # Create coordinate columns
    df['latitude'] = df[region_column].map(lambda x: geocode_cache.get(x, {}).get('latitude'))
    df['longitude'] = df[region_column].map(lambda x: geocode_cache.get(x, {}).get('longitude'))

    return df #.drop(columns=[region_column])

# Usage
wd = geocode_regions(wd, min_delay=2.0)
print("Geocoding completed.")

# Identify rows where geocoding failed (latitude is null) after geocoding
not_found_df = wd[wd['latitude'].isnull()]

# Get a unique list of the region names that were not found
unresolved_regions = not_found_df['RegionName'].unique().tolist()

if unresolved_regions:
    print("The following regions could not be geocoded:")
    for region in unresolved_regions:
        print(f"- {region}")
else:
    print("All regions were successfully geocoded.")


Found 2160 unique regions (0 new)
Geocoding completed.
All regions were successfully geocoded.


In [63]:
unpickled_df = pd.read_pickle("./geocode_cache.pkl")
unpickled_df


{'Serra Gaúcha': {'latitude': -29.1631418, 'longitude': -51.4955259},
 'Campanha Gaúcha': {'latitude': -23.5371755, 'longitude': -46.4168075},
 'Alentejano': {'latitude': -22.9056774, 'longitude': -43.1763427},
 'Santiago': {'latitude': 9.8694792, 'longitude': -83.7980749},
 'Vinho Verde': {'latitude': 41.0674998, 'longitude': -8.2372694},
 'Porto': {'latitude': 41.1494512, 'longitude': -8.6107884},
 'Rio Grande do Sul': {'latitude': -29.8425284, 'longitude': -53.7680577},
 'Vale dos Vinhedos': {'latitude': -29.1790429, 'longitude': -51.5789598},
 'São Paulo': {'latitude': -23.5506507, 'longitude': -46.6333824},
 'Serra Negra': {'latitude': -22.6126459, 'longitude': -46.699986},
 'Santa Catarina': {'latitude': -27.0628367, 'longitude': -51.114965},
 'Vale do São Francisco': {'latitude': -9.4115177, 'longitude': -40.5147187},
 'Campos de Cima da Serra': {'latitude': -28.4299489, 'longitude': -50.897498},
 'Minas Gerais': {'latitude': -18.5264844, 'longitude': -44.1588654},
 'Pinto Bande

In [79]:
region = 'Palmelan'  # or any user input
lat_lon = unpickled_df.get(region, {'latitude': np.nan, 'longitude': np.nan})
latitude, longitude = lat_lon['latitude'], lat_lon['longitude']
latitude, longitude


(nan, nan)

In [77]:
type(wd['Grapes'][0])

str

In [6]:
# Drop the rows where latitude is null
wd['Grapes_list'] = wd['Grapes'].apply(lambda x: eval(x) if isinstance(x, str) else x)
wd = wd.dropna(subset=['latitude', 'longitude'])
wd.head(5)

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,latitude,longitude,Grapes_list
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",-29.163142,-51.495526,[Muscat/Moscato]
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",-29.163142,-51.495526,[Ancellotta]
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",-29.163142,-51.495526,[Cabernet Sauvignon]
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10003,Monte Paschoal,http://www.montepaschoal.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",-29.163142,-51.495526,[Muscat/Moscato]
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",-29.163142,-51.495526,"[Cabernet Sauvignon, Merlot]"


In [7]:
wd.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100646 entries, 0 to 100645
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   WineID       100646 non-null  int64  
 1   WineName     100646 non-null  object 
 2   Type         100646 non-null  object 
 3   Elaborate    100646 non-null  object 
 4   Grapes       100646 non-null  object 
 5   Harmonize    100646 non-null  object 
 6   ABV          100646 non-null  float64
 7   Body         100646 non-null  object 
 8   Acidity      100646 non-null  object 
 9   Code         100646 non-null  object 
 10  Country      100646 non-null  object 
 11  RegionID     100646 non-null  int64  
 12  RegionName   100646 non-null  object 
 13  WineryID     100646 non-null  int64  
 14  WineryName   100646 non-null  object 
 15  Website      82779 non-null   object 
 16  Vintages     100646 non-null  object 
 17  latitude     100646 non-null  float64
 18  longitude    100646 non-

In [8]:
# Load the ratings dataset
ratingstxt = "XWines_Full_21M_ratings.csv"
ratings_file = os.path.join(base_dir, ratingstxt)
ratings = pd.read_csv(ratings_file)
ratings.head()

/var/folders/4h/7hhtxprx1fn6ljyqcdhz50q00000gn/T/ipykernel_3571/3398277075.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv(ratings_file)


,RatingID,UserID,WineID,Vintage,Rating,Date
0,1,1604441,136103,1950,4.0,2019-10-14 11:20:52
1,2,1291483,136103,1950,5.0,2019-11-28 03:36:33
2,3,1070605,104036,1950,5.0,2017-12-28 10:15:55
3,4,1080181,144864,1950,5.0,2016-06-23 02:16:22
4,5,1834379,111430,1950,5.0,2021-05-16 17:58:14


In [9]:
ratings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21013536 entries, 0 to 21013535
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   RatingID  int64  
 1   UserID    int64  
 2   WineID    int64  
 3   Vintage   object 
 4   Rating    float64
 5   Date      object 
dtypes: float64(1), int64(3), object(2)
memory usage: 961.9+ MB


In [10]:
# DATA CLEANING STAGE - Step 1: Assess Data Quality
print("=== DATA QUALITY ASSESSMENT ===")
print("\n1. WINES DATASET:")
print(f"Shape: {wd.shape}")
print(f"Missing values per column:")
print(wd.isnull().sum())
# Create a copy without list columns for duplicate checking
wd_for_dup_check = wd.drop(columns=['Grapes_list'])
print(f"\nDuplicate rows: {wd_for_dup_check.duplicated().sum()}")

print("\n2. RATINGS DATASET:")
print(f"Shape: {ratings.shape}")
print(f"Missing values per column:")
print(ratings.isnull().sum())
print(f"\nDuplicate rows: {ratings.duplicated().sum()}")

# Check data types
print("\n3. WINES DATA TYPES:")
print(wd.dtypes)

=== DATA QUALITY ASSESSMENT ===

1. WINES DATASET:
Shape: (100646, 20)
Missing values per column:
WineID             0
WineName           0
Type               0
Elaborate          0
Grapes             0
Harmonize          0
ABV                0
Body               0
Acidity            0
Code               0
Country            0
RegionID           0
RegionName         0
WineryID           0
WineryName         0
Website        17867
Vintages           0
latitude           0
longitude          0
Grapes_list        0
dtype: int64

Duplicate rows: 0

2. RATINGS DATASET:
Shape: (21013536, 6)
Missing values per column:
RatingID    0
UserID      0
WineID      0
Vintage     0
Rating      0
Date        0
dtype: int64

Duplicate rows: 0

3. WINES DATA TYPES:
WineID           int64
WineName        object
Type            object
Elaborate       object
Grapes          object
Harmonize       object
ABV            float64
Body            object
Acidity         object
Code            object
Country      

In [11]:
# DATA CLEANING STAGE - Step 2: Clean Wine Dataset
print("=== CLEANING WINES DATASET ===")

# Create a clean copy
wines_clean = wd.copy()

# 1. Handle missing values
print("Handling missing values...")
# Drop rows with missing essential information
wines_clean = wines_clean.dropna(subset=['WineName', 'Country', 'Type'])

# Fill missing ABV with median by wine type
wines_clean['ABV'] = wines_clean.groupby('Type')['ABV'].transform(lambda x: x.fillna(x.median()))

# Fill missing RegionName with 'Unknown'
# wines_clean['RegionName'] = wines_clean['RegionName'].fillna('Unknown')

# Fill missing WineryName with 'Unknown'
wines_clean['WineryName'] = wines_clean['WineryName'].fillna('Unknown')

print(f"Rows after cleaning: {len(wines_clean)} (removed {len(wd) - len(wines_clean)} rows)")

# 2. Standardize text fields
print("Standardizing text fields...")
wines_clean['Country'] = wines_clean['Country'].str.strip()
wines_clean['Type'] = wines_clean['Type'].str.strip()
wines_clean['WineName'] = wines_clean['WineName'].str.strip()

# 3. Handle list columns (Grapes, Harmonize, etc.)
print("Processing list columns...")
import ast

def safe_eval(x):
    """Safely evaluate string representations of lists"""
    try:
        if isinstance(x, str):
            return ast.literal_eval(x)
        return x if isinstance(x, list) else []
    except:
        return []

# Apply to list columns
# list_columns = ['Grapes', 'Harmonize', 'Elaborate', 'ABV', 'Body', 'Acidity']
# for col in list_columns:
#     if col in wines_clean.columns:
#         wines_clean[f'{col}_list'] = wines_clean[col].apply(safe_eval)

print("Wine dataset cleaning completed!")

=== CLEANING WINES DATASET ===
Handling missing values...
Rows after cleaning: 100646 (removed 0 rows)
Standardizing text fields...
Processing list columns...
Wine dataset cleaning completed!


In [12]:
# DATA CLEANING STAGE - Step 3: Clean Ratings Dataset
print("=== CLEANING RATINGS DATASET ===")

# Create a clean copy
ratings_clean = ratings.copy()

# 1. Remove invalid ratings
print("Removing invalid ratings...")
initial_count = len(ratings_clean)

# Remove ratings outside valid range (assuming 1-5 scale)
ratings_clean = ratings_clean[(ratings_clean['Rating'] >= 1) & (ratings_clean['Rating'] <= 5)]

# Remove entries with missing UserID or WineID
ratings_clean = ratings_clean.dropna(subset=['UserID', 'WineID'])

print(f"Ratings after cleaning: {len(ratings_clean)} (removed {initial_count - len(ratings_clean)} invalid ratings)")

# 2. Filter ratings for wines that exist in our clean wine dataset
print("Filtering ratings for existing wines...")
valid_wine_ids = set(wines_clean['WineID'])
ratings_clean = ratings_clean[ratings_clean['WineID'].isin(valid_wine_ids)]

print(f"Ratings after filtering: {len(ratings_clean)}")

print("Ratings dataset cleaning completed!")

=== CLEANING RATINGS DATASET ===
Removing invalid ratings...
Ratings after cleaning: 21013536 (removed 0 invalid ratings)
Filtering ratings for existing wines...
Ratings after filtering: 21013536
Ratings dataset cleaning completed!


In [13]:
# # feature selection, before encoding, drop: Elaborate, Code, Country, RegionID, WyneryID, Website, vintages, WineryName
# wd.head(5)

# # WineID is a label, lets drop it and some more...
# wd = wd.drop(columns=['WineID', 'Elaborate', 'Code', 'Country', 'RegionID', 'WineryID', 'Website', 'Vintages', 'WineryName'])


In [14]:
# DATA CLEANING STAGE - Step 4: Prepare Features for k-NN Model
print("=== PREPARING FEATURES FOR k-NN MODEL ===")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
import pandas as pd
import numpy as np
from scipy.sparse import hstack

# Create feature matrix
features_df = wines_clean.copy()

# 1. Process categorical variables
print("Processing categorical variables...")

# a. OneHotEncode Type column
print("- One-hot encoding wine types...")
type_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
type_encoded = type_encoder.fit_transform(features_df[['Type']])
type_columns = [f'Type_{cat}' for cat in type_encoder.categories_[0]]
type_df = pd.DataFrame(type_encoded, columns=type_columns, index=features_df.index)

# b. Process Grapes (extract top N grapes)
print("- Processing grape varieties...")
N_TOP_GRAPES = 60  # Consider top 60 grape varieties

# Get most common grape varieties
all_grapes = [grape for sublist in features_df['Grapes_list'] for grape in sublist if isinstance(sublist, list)]
top_grapes = pd.Series(all_grapes).value_counts().head(N_TOP_GRAPES).index.tolist()

# Create binary columns for each top grape
for grape in top_grapes:
    features_df[f'Grape_{grape}'] = features_df['Grapes_list'].apply(
        lambda x: 1 if isinstance(x, list) and grape in x else 0
    )

grape_columns = [f'Grape_{grape}' for grape in top_grapes]

# c. Ordinal encode Body and Acidity
print("- Ordinal encoding Body and Acidity...")
# Define the order for Body
body_categories = ['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']
body_encoder = OrdinalEncoder(categories=[body_categories])
features_df['Body_encoded'] = np.nan
body_mask = features_df['Body'].isin(body_categories)
if body_mask.any():
    features_df.loc[body_mask, 'Body_encoded'] = body_encoder.fit_transform(
        features_df.loc[body_mask, ['Body']])
features_df['Body_encoded'] = features_df['Body_encoded'].fillna(2)  # Default to medium-bodied

# Define the order for Acidity
acidity_categories = ['Low', 'Medium', 'High']
acidity_encoder = OrdinalEncoder(categories=[acidity_categories])
features_df['Acidity_encoded'] = np.nan
acidity_mask = features_df['Acidity'].isin(acidity_categories)
if acidity_mask.any():
    features_df.loc[acidity_mask, 'Acidity_encoded'] = acidity_encoder.fit_transform(
        features_df.loc[acidity_mask, ['Acidity']])
features_df['Acidity_encoded'] = features_df['Acidity_encoded'].fillna(1)  # Default to medium acidity

# 2. Handle numerical features
print("Processing numerical features...")
# Ensure ABV is numeric
features_df['ABV_numeric'] = pd.to_numeric(features_df['ABV'], errors='coerce')
features_df['ABV_numeric'] = features_df['ABV_numeric'].fillna(features_df['ABV_numeric'].median())

# Keep latitude and longitude as is (they're already numeric)
numeric_columns = ['ABV_numeric', 'latitude', 'longitude']

# 3. Create wine profile features from aggregated ratings
print("Creating wine profile features...")
wine_stats = ratings_clean.groupby('WineID').agg({
    'Rating': ['mean', 'count', 'std']
}).round(2)

wine_stats.columns = ['avg_rating', 'rating_count', 'rating_std']
wine_stats = wine_stats.fillna(0)

# Merge with wine features
features_df = features_df.merge(wine_stats, left_on='WineID', right_index=True, how='left')
features_df[['avg_rating', 'rating_count', 'rating_std']] = features_df[['avg_rating', 'rating_count', 'rating_std']].fillna(0)
rating_columns = ['avg_rating', 'rating_count', 'rating_std']

# 4. Select final feature columns for k-NN
print("Compiling final feature set...")
# Combine all feature columns
feature_columns = numeric_columns + ['Body_encoded', 'Acidity_encoded'] + rating_columns + grape_columns
feature_dfs = [features_df[feature_columns], type_df]

# Create final feature matrix
X = pd.concat(feature_dfs, axis=1)

# 5. Scale features
print("Scaling features...")
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=features_df.index)

print(f"Feature matrix shape: {X_scaled_df.shape}")
print(f"Feature columns: {len(X_scaled_df.columns)} columns")
print("\nFeature matrix ready for k-NN model!")

# Update the feature_columns variable to include all columns
feature_columns = X_scaled_df.columns.tolist()

=== PREPARING FEATURES FOR k-NN MODEL ===
Processing categorical variables...
- One-hot encoding wine types...
- Processing grape varieties...
- Ordinal encoding Body and Acidity...
Processing numerical features...
Creating wine profile features...
Compiling final feature set...
Scaling features...
Feature matrix shape: (100646, 74)
Feature columns: 74 columns

Feature matrix ready for k-NN model!


In [42]:
X_scaled_df.columns

Index(['ABV_numeric', 'latitude', 'longitude', 'Body_encoded',
       'Acidity_encoded', 'avg_rating', 'rating_count', 'rating_std',
       'Grape_Cabernet Sauvignon', 'Grape_Chardonnay', 'Grape_Merlot',
       'Grape_Pinot Noir', 'Grape_Syrah/Shiraz', 'Grape_Cabernet Franc',
       'Grape_Grenache', 'Grape_Sauvignon Blanc', 'Grape_Riesling',
       'Grape_Malbec', 'Grape_Sangiovese', 'Grape_Tempranillo',
       'Grape_Touriga Nacional', 'Grape_Mourvedre', 'Grape_Petit Verdot',
       'Grape_Nebbiolo', 'Grape_Corvina', 'Grape_Viognier', 'Grape_Zinfandel',
       'Grape_Tinta Roriz', 'Grape_Glera/Prosecco', 'Grape_Touriga Franca',
       'Grape_Rondinella', 'Grape_Carmenère', 'Grape_Sémillon',
       'Grape_Chenin Blanc', 'Grape_Barbera', 'Grape_Garnacha',
       'Grape_Carignan/Cariñena', 'Grape_Cinsault', 'Grape_Pinot Meunier',
       'Grape_Gewürztraminer', 'Grape_Pinot Blanc', 'Grape_Pinot Grigio',
       'Grape_Primitivo', 'Grape_Spätburgunder', 'Grape_Montepulciano',
       'Grape

In [15]:
# Extract unique categories from Body and Acidity columns

# acidity_categories = ['Low', 'Medium', 'High']  # Filter out non-string categories
# body_categories = ['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']  # Filter out non-string categories

print(acidity_categories)
print(body_categories)
features_df.head()


['Low', 'Medium', 'High']
['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']


,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,...,Grape_Garganega,Grape_Muscat Blanc,Grape_Corvinone,Grape_Grauburgunder,Body_encoded,Acidity_encoded,ABV_numeric,avg_rating,rating_count,rating_std
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,...,0,0,0,0,2.0,2.0,7.5,4.13,2625,0.74
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,...,0,0,0,0,2.0,1.0,12.0,3.50,10,1.00
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,...,0,0,0,0,3.0,2.0,12.0,3.38,62,0.71
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,BR,...,0,0,0,0,2.0,1.0,12.0,3.17,110,0.81
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,...,0,0,0,0,3.0,1.0,11.0,2.97,72,0.97


In [16]:
# DATA CLEANING STAGE - Step 5: Build k-NN Model
print("=== BUILDING k-NN RECOMMENDATION MODEL ===")

from sklearn.neighbors import NearestNeighbors
import joblib

# 1. Initialize and train k-NN model
print("Training k-NN model...")
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # 6 to get 5 recommendations (excluding self)
knn_model.fit(X_scaled_df)

# 2. Create recommendation function
def get_wine_recommendations(wine_id, n_recommendations=5):
    """Get wine recommendations based on k-NN similarity"""
    try:
        # Find the index of the wine
        wine_idx = features_df[features_df['WineID'] == wine_id].index[0]

        # Get the wine's feature vector
        wine_features = X_scaled_df.loc[wine_idx].values.reshape(1, -1)

        # Find similar wines
        distances, indices = knn_model.kneighbors(wine_features, n_neighbors=n_recommendations+1)

        # Get recommended wine IDs (excluding the input wine itself)
        recommended_indices = indices[0][1:]  # Skip the first one (itself)
        recommended_wine_ids = features_df.iloc[recommended_indices]['WineID'].tolist()

        return recommended_wine_ids, distances[0][1:]
    except IndexError:
        return [], []

# 3. Test the model
print("Testing k-NN model...")
test_wine_id = features_df['WineID'].iloc[0]  # Use first wine as test
recommendations, distances = get_wine_recommendations(test_wine_id)

print(f"Test wine ID: {test_wine_id}")
print(f"Recommended wine IDs: {recommendations}")
print(f"Distances: {distances}")

# Display test results
if recommendations:
    test_wine_info = features_df[features_df['WineID'] == test_wine_id][['WineName', 'Country', 'Type']].iloc[0]
    print(f"\nOriginal wine: {test_wine_info['WineName']} ({test_wine_info['Country']}, {test_wine_info['Type']})")

    print("\nRecommended wines:")
    for i, rec_id in enumerate(recommendations):
        rec_info = features_df[features_df['WineID'] == rec_id][['WineName', 'Country', 'Type']].iloc[0]
        print(f"{i+1}. {rec_info['WineName']} ({rec_info['Country']}, {rec_info['Type']}) - Distance: {distances[i]:.3f}")

print("\nk-NN model successfully built and tested!")

=== BUILDING k-NN RECOMMENDATION MODEL ===
Training k-NN model...
Testing k-NN model...
Test wine ID: 100001
Recommended wine IDs: [100061, 100676, 101336, 100065, 100828]
Distances: [0.00057118 0.00127446 0.00131561 0.00149189 0.0015169 ]

Original wine: Espumante Moscatel (Brazil, Sparkling)

Recommended wines:
1. Moscatel (Brazil, Sparkling) - Distance: 0.001
2. Moscatel Rosé (Brazil, Sparkling) - Distance: 0.001
3. Moscatel Espumante (Brazil, Sparkling) - Distance: 0.001
4. Moscatel Espumante (Brazil, Sparkling) - Distance: 0.001
5. Moscatel Espumante (Brazil, Sparkling) - Distance: 0.002

k-NN model successfully built and tested!


/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
# # DATA CLEANING STAGE - Step 6: Save Cleaned Data and Model
# print("=== SAVING CLEANED DATA AND MODEL ===")

# import os
# import joblib

# # Create data directory if it doesn't exist
# data_dir = "/Users/mjh/code/Obispodino/cvino/data"
# os.makedirs(data_dir, exist_ok=True)

# # 1. Save cleaned datasets
# print("Saving cleaned datasets...")
# wines_clean.to_csv(os.path.join(data_dir, 'wines_clean.csv'), index=False)
# ratings_clean.to_csv(os.path.join(data_dir, 'ratings_clean.csv'), index=False)
# features_df.to_csv(os.path.join(data_dir, 'wine_features.csv'), index=False)

# # 2. Create a dictionary of all encoders used
# label_encoders = {
#     'type_encoder': type_encoder,
#     'body_encoder': body_encoder,
#     'acidity_encoder': acidity_encoder
# }

# # 3. Save the trained model and preprocessing objects
# print("Saving k-NN model and preprocessing objects...")
# model_data = {
#     'knn_model': knn_model,
#     'scaler': scaler,
#     'label_encoders': label_encoders,
#     'feature_columns': feature_columns,
#     'wine_features': features_df
# }

# joblib.dump(model_data, os.path.join(data_dir, 'wine_recommendation_model.joblib'))

# print("All data and model files saved successfully!")
# print("\nFiles created:")
# print(f"- {data_dir}/wines_clean.csv")
# print(f"- {data_dir}/ratings_clean.csv")
# print(f"- {data_dir}/wine_features.csv")
# print(f"- {data_dir}/wine_recommendation_model.joblib")

# print("\n=== DATA CLEANING STAGE COMPLETED ===")
# print("Ready to proceed with backend development!")

=== SAVING CLEANED DATA AND MODEL ===
Saving cleaned datasets...
Saving k-NN model and preprocessing objects...
All data and model files saved successfully!

Files created:
- /Users/mjh/code/Obispodino/cvino/data/wines_clean.csv
- /Users/mjh/code/Obispodino/cvino/data/ratings_clean.csv
- /Users/mjh/code/Obispodino/cvino/data/wine_features.csv
- /Users/mjh/code/Obispodino/cvino/data/wine_recommendation_model.joblib

=== DATA CLEANING STAGE COMPLETED ===
Ready to proceed with backend development!


In [ ]:
# # Print a comprehensive summary of the wine recommendation model

# print("=== WINE RECOMMENDATION MODEL SUMMARY ===\n")

# # 1. Model Overview
# print("MODEL OVERVIEW:")
# print(f"- Type: k-Nearest Neighbors (k-NN)")
# print(f"- Algorithm: {knn_model.algorithm}")
# print(f"- Distance metric: {knn_model.metric}")
# print(f"- Number of neighbors: {knn_model.n_neighbors}")
# print(f"- Total wines in dataset: {len(features_df)}")

# # 2. Feature Information
# print("\nFEATURE INFORMATION:")
# print(f"- Number of features: {len(feature_columns)}")
# print("- Features used:")
# for col in feature_columns:
#     print(f"  • {col}")

# # 3. Dataset Statistics
# print("\nDATASET STATISTICS:")
# print(f"- Total unique countries: {features_df['Country'].nunique()}")
# print(f"- Total unique wine types: {features_df['Type'].nunique()}")
# print(f"- Total unique regions: {features_df['RegionName'].nunique()}")
# print(f"- Total unique wineries: {features_df['WineryName'].nunique()}")
# print(f"- Average rating across all wines: {features_df['avg_rating'].mean():.2f}")
# print(f"- Average number of ratings per wine: {features_df['rating_count'].mean():.2f}")

# # 4. Wine Type Distribution
# type_dist = features_df['Type'].value_counts()
# print("\nWINE TYPE DISTRIBUTION:")
# for wine_type, count in type_dist.items():
#     print(f"- {wine_type}: {count} wines ({count/len(features_df)*100:.1f}%)")

# # 5. Model Usage Example
# print("\nMODEL USAGE EXAMPLE:")
# print("To get wine recommendations:")
# print("```python")
# print("wine_id = 100001  # Example wine ID")
# print("recommendations, distances = get_wine_recommendations(wine_id, n_recommendations=5)")
# print("```")

# print("\nRECOMMENDATION METHODOLOGY:")
# print("1. The model uses cosine similarity to find wines with similar characteristics")
# print("2. Features are scaled to ensure equal importance")
# print("3. Categorical variables are encoded to numerical values")
# print("4. Wine profile features combine intrinsic characteristics with user ratings")
# print("5. The model returns the most similar wines based on these features")

# print("\n=== END OF MODEL SUMMARY ===")

=== WINE RECOMMENDATION MODEL SUMMARY ===

MODEL OVERVIEW:
- Type: k-Nearest Neighbors (k-NN)
- Algorithm: brute
- Distance metric: cosine
- Number of neighbors: 6
- Total wines in dataset: 100646

FEATURE INFORMATION:
- Number of features: 74
- Features used:
  • ABV_numeric
  • latitude
  • longitude
  • Body_encoded
  • Acidity_encoded
  • avg_rating
  • rating_count
  • rating_std
  • Grape_Cabernet Sauvignon
  • Grape_Chardonnay
  • Grape_Merlot
  • Grape_Pinot Noir
  • Grape_Syrah/Shiraz
  • Grape_Cabernet Franc
  • Grape_Grenache
  • Grape_Sauvignon Blanc
  • Grape_Riesling
  • Grape_Malbec
  • Grape_Sangiovese
  • Grape_Tempranillo
  • Grape_Touriga Nacional
  • Grape_Mourvedre
  • Grape_Petit Verdot
  • Grape_Nebbiolo
  • Grape_Corvina
  • Grape_Viognier
  • Grape_Zinfandel
  • Grape_Tinta Roriz
  • Grape_Glera/Prosecco
  • Grape_Touriga Franca
  • Grape_Rondinella
  • Grape_Carmenère
  • Grape_Sémillon
  • Grape_Chenin Blanc
  • Grape_Barbera
  • Grape_Garnacha
  • Grape_Cari

In [19]:

print("\nk-NN model successfully built and tested!")

# CELL FOR EVALUATION SETUP
print("\n=== MODEL EVALUATION SETUP ===")
import random
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import numpy as np
import pandas as pd

# Ensure features_df has WineID as a readily accessible column and is indexed appropriately
# If WineID is not the index, set it for easy lookup if needed, or ensure efficient filtering.
# For this example, we'll assume features_df can be filtered by WineID easily.

def get_wine_details(wine_id_list):
    """Helper function to get details for a list of wine IDs."""
    if not wine_id_list:
        return pd.DataFrame()
    return features_df[features_df['WineID'].isin(wine_id_list)][['WineID', 'WineName', 'Country', 'Type', 'RegionName', 'Body', 'Acidity', 'Grapes_list', 'avg_rating']]

def jaccard_similarity(list1, list2):
    """Calculates Jaccard similarity between two lists."""
    if not isinstance(list1, list) or not isinstance(list2, list):
        return 0.0
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0.0

print("Evaluation helper functions defined.")


k-NN model successfully built and tested!

=== MODEL EVALUATION SETUP ===
Evaluation helper functions defined.


In [20]:
# CELL FOR ENHANCED QUALITATIVE ANALYSIS
print("\n=== ENHANCED QUALITATIVE ANALYSIS ===")

# Select a diverse set of test_wine_ids
# Example: choose a few randomly, or pick specific ones based on characteristics
num_test_wines_qualitative = 5
if len(features_df) >= num_test_wines_qualitative:
    # Ensure we have 'WineID' column in features_df
    if 'WineID' in features_df.columns:
        sample_wine_ids = random.sample(features_df['WineID'].tolist(), min(num_test_wines_qualitative, len(features_df)))
    else:
        print("Error: 'WineID' column not found in features_df.")
        sample_wine_ids = []
else:
    sample_wine_ids = features_df['WineID'].tolist() if 'WineID' in features_df.columns else []


for test_id in sample_wine_ids:
    print(f"\n--- Qualitative Test for WineID: {test_id} ---")
    original_wine_info = get_wine_details([test_id])
    if original_wine_info.empty:
        print("Original wine not found.")
        continue

    print("\nOriginal Wine:")
    print(original_wine_info.to_string())

    recommendations, distances = get_wine_recommendations(test_id, n_recommendations=5)

    if recommendations:
        print("\nRecommended Wines:")
        recommended_wines_info = get_wine_details(recommendations)
        for i, rec_id in enumerate(recommendations):
            rec_info_row = recommended_wines_info[recommended_wines_info['WineID'] == rec_id]
            print(f"\n{i+1}. WineID: {rec_id} (Distance: {distances[i]:.4f})")
            print(rec_info_row.to_string())
    else:
        print("No recommendations found.")
    print("-" * 50)


=== ENHANCED QUALITATIVE ANALYSIS ===

--- Qualitative Test for WineID: 102444 ---

Original Wine:
      WineID          WineName   Country   Type RegionName           Body Acidity                          Grapes_list  avg_rating
2442  102444  Alabastro Branco  Portugal  White    Setúbal  Medium-bodied    High  [Fernão Pires, Antão Vaz, Roupeiro]        3.27

Recommended Wines:

1. WineID: 102451 (Distance: 0.0000)
      WineID         WineName   Country   Type RegionName           Body Acidity                                                    Grapes_list  avg_rating
2449  102451  Alentejo Branco  Portugal  White   Alentejo  Medium-bodied    High  [Arinto de Bucelas, Rabo de Ovelha, Antão Vaz, Perrum, Síria]         3.3

2. WineID: 102593 (Distance: 0.0001)
      WineID       WineName   Country   Type RegionName           Body Acidity          Grapes_list  avg_rating
2591  102593  Lisboa Arinto  Portugal  White     Lisboa  Medium-bodied    High  [Arinto de Bucelas]        3.29

3. Wi

/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvi

In [21]:
# CELL FOR QUANTITATIVE FEATURE AGREEMENT & SIMILARITY
print("\n=== QUANTITATIVE FEATURE AGREEMENT & SIMILARITY ===")

def evaluate_recommendation_features(input_wine_id, recommended_ids):
    if not recommended_ids:
        return None

    input_wine_details = features_df[features_df['WineID'] == input_wine_id].iloc[0]
    recommended_wines_details = features_df[features_df['WineID'].isin(recommended_ids)]

    metrics = {
        'type_match_percentage': 0,
        'country_match_percentage': 0,
        'avg_body_diff': np.nan, # Using encoded body for difference
        'avg_acidity_diff': np.nan, # Using encoded acidity for difference
        'avg_grape_jaccard': 0
    }

    type_matches = 0
    country_matches = 0
    body_diffs = []
    acidity_diffs = []
    grape_jaccards = []

    # Get encoded values for body and acidity for the input wine
    # Ensure 'Body_encoded' and 'Acidity_encoded' exist from Step 4
    input_body_encoded = input_wine_details.get('Body_encoded', np.nan)
    input_acidity_encoded = input_wine_details.get('Acidity_encoded', np.nan)

    for _, rec_wine in recommended_wines_details.iterrows():
        if rec_wine['Type'] == input_wine_details['Type']:
            type_matches += 1
        if rec_wine['Country'] == input_wine_details['Country']:
            country_matches += 1

        rec_body_encoded = rec_wine.get('Body_encoded', np.nan)
        rec_acidity_encoded = rec_wine.get('Acidity_encoded', np.nan)

        if not np.isnan(input_body_encoded) and not np.isnan(rec_body_encoded):
            body_diffs.append(abs(rec_body_encoded - input_body_encoded))
        if not np.isnan(input_acidity_encoded) and not np.isnan(rec_acidity_encoded):
            acidity_diffs.append(abs(rec_acidity_encoded - input_acidity_encoded))

        grape_jaccards.append(jaccard_similarity(input_wine_details['Grapes_list'], rec_wine['Grapes_list']))

    num_recommendations = len(recommended_ids)
    metrics['type_match_percentage'] = (type_matches / num_recommendations) * 100
    metrics['country_match_percentage'] = (country_matches / num_recommendations) * 100
    if body_diffs:
        metrics['avg_body_diff'] = np.mean(body_diffs)
    if acidity_diffs:
        metrics['avg_acidity_diff'] = np.mean(acidity_diffs)
    if grape_jaccards:
        metrics['avg_grape_jaccard'] = np.mean(grape_jaccards)

    return metrics

# Test with a sample of wines
num_test_wines_quantitative = 10
all_feature_eval_metrics = []

if len(features_df) >= num_test_wines_quantitative:
    if 'WineID' in features_df.columns:
        quantitative_sample_ids = random.sample(features_df['WineID'].tolist(), min(num_test_wines_quantitative, len(features_df)))
    else:
        print("Error: 'WineID' column not found in features_df for quantitative test.")
        quantitative_sample_ids = []
else:
    quantitative_sample_ids = features_df['WineID'].tolist() if 'WineID' in features_df.columns else []


for test_id in quantitative_sample_ids:
    recommendations, _ = get_wine_recommendations(test_id, n_recommendations=5)
    if recommendations:
        eval_metrics = evaluate_recommendation_features(test_id, recommendations)
        if eval_metrics:
            eval_metrics['input_wine_id'] = test_id
            all_feature_eval_metrics.append(eval_metrics)
            # print(f"Metrics for WineID {test_id}: {eval_metrics}")


if all_feature_eval_metrics:
    feature_eval_summary_df = pd.DataFrame(all_feature_eval_metrics)
    print("\n--- Feature Agreement & Similarity Summary ---")
    print(feature_eval_summary_df.mean(numeric_only=True)) # Display average metrics
    # print("\nDetailed Metrics per Wine:")
    # print(feature_eval_summary_df.to_string())
else:
    print("No quantitative feature evaluation metrics to display.")


=== QUANTITATIVE FEATURE AGREEMENT & SIMILARITY ===


/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvi


--- Feature Agreement & Similarity Summary ---
type_match_percentage          100.000000
country_match_percentage        82.000000
avg_body_diff                    0.000000
avg_acidity_diff                 0.000000
avg_grape_jaccard                0.913333
input_wine_id               147143.100000
dtype: float64


/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.p

In [60]:
distances, indices

(array([[0.        , 0.00057118, 0.00127446, 0.00131561, 0.00149189,
         0.0015169 ]]),
 array([[   0,   60,  675, 1335,   64,  827]]))

In [ ]:
# Test the model with some features instead of WineID
test_features = X_scaled_df.iloc[0].values.reshape(1, -1)  # Use the first wine's features
distances, indices = knn_model.kneighbors(test_features, n_neighbors=6)  # 6 to get 5 recommendations (excluding self)
recommended_wine_ids = features_df.iloc[indices[0][1:]]['WineID'].tolist()  # Skip the first one (itself)
print("\n--- Test with Features ---")
print(f"Test features: {test_features.flatten()}")
print(f"Recommended wine IDs: {recommended_wine_ids}")
# Display the recommended wines based on features
recommended_wines_info = get_wine_details(recommended_wine_ids)
print("\nRecommended Wines:")
print(recommended_wines_info.to_string(index=False))



--- Test with Features ---
Test features: [0.15       0.14710864 0.32551398 0.5        1.         0.768
 0.09558555 0.38541667 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         1.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.         0.        ]
Recommended wine IDs: [100061, 100676, 101336, 100065, 100828]

Recommended Wines:
 WineID           WineName Country      Type        RegionName          Body Acidity      Grapes_list  avg_rating
 100061           Moscate

/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
# SIMPLE WINE RECOMMENDATION FUNCTION ,WORKING AND DEFINED
import pandas as pd
import numpy as np

def get_wine_recommendations_by_characteristics(
    wine_type='Red',                   # Wine type (Red, White, Rosé, Sparkling, etc.)
    grape_varieties='Merlot',          # Grape varieties (single string or list)
    body='Medium-bodied',              #['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']
    abv=14.0,                          # Alcohol by volume percentage
    acidity=None,                      # Optional - Acidity (Low, Medium, High)
    country=None,                      # Optional - Country of origin
    region_name=None,                  # Optional - Region name
    n_recommendations=5                # Number of recommendations to return
):
    """Get wine recommendations based on specified characteristics."""
    # Initialize feature vector
    input_features = {}

    # Process wine type (one-hot encoding)
    if wine_type in type_encoder.categories_[0]:
        type_vector = type_encoder.transform([[wine_type]])[0]
        for i, col in enumerate(type_columns):
            input_features[col] = type_vector[i]
    else:
        # Default to most common type if provided type not found
        print(f"Warning: Wine type '{wine_type}' not found in dataset. Using default encoding.")
        for col in type_columns:
            input_features[col] = 0

    # Process grape varieties (binary features)
    if isinstance(grape_varieties, str):
        # Convert comma-separated string to list
        if ',' in grape_varieties:
            grape_varieties = [g.strip() for g in grape_varieties.split(',')]
        else:
            grape_varieties = [grape_varieties]

    # Initialize all grape features to 0
    for col in grape_columns:
        input_features[col] = 0

    # Set matching grape features to 1
    grape_found = False
    for grape in grape_varieties:
        col_name = f'Grape_{grape}'
        if col_name in grape_columns:
            input_features[col_name] = 1
            grape_found = True

    if not grape_found:
        print(f"Warning: None of the specified grape varieties {grape_varieties} found in dataset.")

    # Process body (ordinal encoding)
    if body in body_categories:
        body_encoded = body_encoder.transform([[body]])[0][0]
    else:
        body_encoded = 2  # Default to medium-bodied
        print(f"Warning: Body type '{body}' not recognized. Using medium-bodied as default.")
    input_features['Body_encoded'] = body_encoded

    # Process acidity (ordinal encoding) - only if provided
    if acidity in acidity_categories:
        acidity_encoded = acidity_encoder.transform([[acidity]])[0][0]
    else:
        acidity_encoded = 1  # Default to medium acidity
    input_features['Acidity_encoded'] = acidity_encoded

    # Process ABV
    try:
        input_features['ABV_numeric'] = float(abv)
    except (ValueError, TypeError):
        input_features['ABV_numeric'] = features_df['ABV_numeric'].median()
        print(f"Warning: Invalid ABV value. Using median ({input_features['ABV_numeric']:.1f}) instead.")

    # Process geographic information (region and country)
    if region_name:
        # Look for matching region in the dataset
        matching_regions = features_df[features_df['RegionName'] == region_name]
        if not matching_regions.empty:
            # Use the latitude/longitude of the matching region
            input_features['latitude'] = matching_regions['latitude'].iloc[0]
            input_features['longitude'] = matching_regions['longitude'].iloc[0]
        else:
            print(f"Warning: Region '{region_name}' not found in dataset. Using default coordinates.")
            input_features['latitude'] = features_df['latitude'].median()
            input_features['longitude'] = features_df['longitude'].median()
    else:
        # If no region specified, use median values
        input_features['latitude'] = features_df['latitude'].median()
        input_features['longitude'] = features_df['longitude'].median()

    # Set default values for remaining features
    input_features['avg_rating'] = features_df['avg_rating'].median()
    input_features['rating_count'] = 0
    input_features['rating_std'] = 0

    # Create feature DataFrame and get recommendations
    input_df = pd.DataFrame([{col: input_features.get(col, 0) for col in feature_columns}])

    # Ensure all required columns are present
    missing_cols = [col for col in feature_columns if col not in input_df.columns]
    if missing_cols:
        print(f"Warning: Missing columns in input: {missing_cols}")
        for col in missing_cols:
            input_df[col] = 0

    # Get extra recommendations to account for filtering
    n_neighbors = max(n_recommendations * 3, 20) if country else n_recommendations

    try:
        # Scale the input features
        scaled_input = scaler.transform(input_df)

        # Get recommendations from the model
        distances, indices = knn_model.kneighbors(scaled_input, n_neighbors=n_neighbors)

        # Retrieve wine IDs
        recommended_indices = indices[0]
        recommended_wine_ids = features_df.iloc[recommended_indices]['WineID'].tolist()

        # Get full wine details
        recommended_wines = features_df[features_df['WineID'].isin(recommended_wine_ids)][
            ['WineID', 'WineName', 'Country', 'Type', 'RegionName', 'Body', 'Acidity', 'Grapes_list', 'avg_rating']
        ]

        # Add similarity scores
        distance_map = {wine_id: dist for wine_id, dist in zip(recommended_wine_ids, distances[0])}
        recommended_wines['Similarity'] = recommended_wines['WineID'].map(lambda wine_id: 1 - distance_map[wine_id])

        # Filter by country if specified
        if country:
            country_matches = recommended_wines[recommended_wines['Country'] == country]
            if not country_matches.empty:
                recommended_wines = country_matches
            else:
                print(f"Warning: No wines found from '{country}'. Showing wines from all countries.")

        # Sort by similarity and limit results
        result = recommended_wines.sort_values('Similarity', ascending=False).head(n_recommendations)

        if result.empty:
            print("No matching wines found. Try different criteria.")
            return pd.DataFrame()

        return result

    except Exception as e:
        print(f"Error getting recommendations: {str(e)}")
        return pd.DataFrame()

# Example usage
recommendations = get_wine_recommendations_by_characteristics(

    wine_type='Red',                   # Wine type (Red, White, Rosé, Sparkling, Dessert, Dessert/Port etc.)
    grape_varieties='Malbec',          # Grape varieties (single string or list)
    body='Full-bodied',              #['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']
    abv=12.0,                          # Alcohol by volume percentage, min 0, max 50
    acidity=None,                      # Optional - Acidity (Low, Medium, High)
    country="Argentina",                      # Optional - Country of origin
    region_name=None,                  # Optional - Region name
    n_recommendations=9                # Number of recommendations to return

)

# Display recommendations
if not recommendations.empty:
    print("Recommended wines based on your preferences:")
    for i, (_, wine) in enumerate(recommendations.iterrows(), 1):
        grapes_str = ', '.join(wine['Grapes_list']) if isinstance(wine['Grapes_list'], list) else 'Unknown'
        print(f"{i}. {wine['WineName']} ({wine['Country']}, {wine['Type']})")
        print(f"   Region: {wine['RegionName'] or 'Unknown'}")
        print(f"   Grapes: {grapes_str}")
        print(f"   Body: {wine['Body'] or 'Unknown'}, Acidity: {wine['Acidity'] or 'Unknown'}")
        print(f"   Rating: {wine['avg_rating']:.1f}/5")
        print(f"   Similarity: {wine['Similarity']:.2f}")
        print()
else:
    print("No recommendations found. Try adjusting your preferences.")


Recommended wines based on your preferences:
1. Mora Negra (Argentina, Red)
   Region: San Juan
   Grapes: Malbec, Bonarda
   Body: Full-bodied, Acidity: Medium
   Rating: 4.1/5
   Similarity: 0.98

2. Zeta La Ribera Malbec (Argentina, Red)
   Region: Paraje Altamira
   Grapes: Malbec
   Body: Very full-bodied, Acidity: High
   Rating: 4.2/5
   Similarity: 0.97

3. Malbec Superior (Argentina, Red)
   Region: Paraje Altamira
   Grapes: Malbec
   Body: Very full-bodied, Acidity: High
   Rating: 4.1/5
   Similarity: 0.97

4. Premium Reserve Malbec-Bonarda (Argentina, Red)
   Region: San Rafael
   Grapes: Malbec, Bonarda
   Body: Full-bodied, Acidity: Medium
   Rating: 4.2/5
   Similarity: 0.97

5. Finca El Tomillo Single Vineyard Malbec (Argentina, Red)
   Region: Paraje Altamira
   Grapes: Malbec
   Body: Very full-bodied, Acidity: High
   Rating: 4.3/5
   Similarity: 0.97



/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret

In [61]:
recommendations

,WineID,WineName,Country,Type,RegionName,Body,Acidity,Grapes_list,avg_rating,Harmonize,Similarity
48407,148459,Prima Pietra Appassimento Leggero Rosso,Italy,Red,Veneto,Full-bodied,High,"[Merlot, Corvina]",3.87,"[Beef, Pasta, Lamb, Game Meat]",0.976035
43481,143533,Mille 1 Garda Rosso,Italy,Red,Garda,Full-bodied,High,"[Rebo, Merlot, Corvina]",3.90,"[Beef, Pasta, Lamb, Game Meat]",0.972694
38561,138613,Rosso Verona,Italy,Red,Rosso Veronese,Full-bodied,High,"[Corvina, Merlot]",3.68,"[Beef, Pasta, Lamb, Game Meat]",0.972317
40791,140843,Cormì Corvina-Merlot,Italy,Red,Veneto,Full-bodied,High,"[Corvina, Merlot]",3.78,"[Beef, Pasta, Lamb, Game Meat]",0.971787
36910,136962,Rosso Veneto,Italy,Red,Veneto,Full-bodied,High,"[Merlot, Corvina]",3.94,"[Beef, Pasta, Lamb, Game Meat]",0.971410
40118,140170,Appassimento Parziale,Italy,Red,Veneto,Full-bodied,High,"[Corvina, Merlot]",3.81,"[Beef, Pasta, Lamb, Game Meat]",0.971250
44600,144652,Cà Nova,Italy,Red,Valpolicella,Full-bodied,High,"[Merlot, Corvina]",3.72,"[Beef, Pasta, Lamb, Game Meat]",0.971183
40975,141027,Gran Passione,Italy,Red,Veneto,Full-bodied,High,"[Merlot, Corvina]",3.92,"[Beef, Pasta, Lamb, Game Meat]",0.970501
42234,142286,Raudii Corvina-Merlot,Italy,Red,Veneto,Full-bodied,High,"[Merlot, Corvina]",3.53,"[Beef, Pasta, Lamb, Game Meat]",0.969708


In [58]:
# WINE RECOMMENDATION FUNCTION WITH FOOD PAIRINGS
import pandas as pd
import numpy as np

def get_wine_recommendations_by_characteristics(
    wine_type='Red',                   # Wine type (Red, White, Rosé, Sparkling, etc.)
    grape_varieties='Merlot',          # Grape varieties (single string or list)
    body='Medium-bodied',              # Body (Very light to Very full-bodied)
    abv=14.0,                          # Alcohol by volume percentage
    acidity=None,                      # Optional - Acidity (Low, Medium, High)
    country=None,                      # Optional - Country of origin
    region_name=None,                  # Optional - Region name
    n_recommendations=5                # Number of recommendations to return
):
    """Get wine recommendations based on specified characteristics."""
    # Initialize feature vector
    input_features = {}

    # Process wine type (one-hot encoding)
    if wine_type in type_encoder.categories_[0]:
        type_vector = type_encoder.transform([[wine_type]])[0]
        for i, col in enumerate(type_columns):
            input_features[col] = type_vector[i]
    else:
        # Default to most common type if provided type not found
        print(f"Warning: Wine type '{wine_type}' not found in dataset. Using default encoding.")
        for col in type_columns:
            input_features[col] = 0

    # Process grape varieties (binary features)
    if isinstance(grape_varieties, str):
        # Convert comma-separated string to list
        if ',' in grape_varieties:
            grape_varieties = [g.strip() for g in grape_varieties.split(',')]
        else:
            grape_varieties = [grape_varieties]

    # Initialize all grape features to 0
    for col in grape_columns:
        input_features[col] = 0

    # Set matching grape features to 1
    grape_found = False
    for grape in grape_varieties:
        col_name = f'Grape_{grape}'
        if col_name in grape_columns:
            input_features[col_name] = 1
            grape_found = True

    if not grape_found:
        print(f"Warning: None of the specified grape varieties {grape_varieties} found in dataset.")

    # Process body (ordinal encoding)
    if body in body_categories:
        body_encoded = body_encoder.transform([[body]])[0][0]
    else:
        body_encoded = 2  # Default to medium-bodied
        print(f"Warning: Body type '{body}' not recognized. Using medium-bodied as default.")
    input_features['Body_encoded'] = body_encoded

    # Process acidity (ordinal encoding) - only if provided
    if acidity in acidity_categories:
        acidity_encoded = acidity_encoder.transform([[acidity]])[0][0]
    else:
        acidity_encoded = 1  # Default to medium acidity
    input_features['Acidity_encoded'] = acidity_encoded

    # Process ABV
    try:
        input_features['ABV_numeric'] = float(abv)
    except (ValueError, TypeError):
        input_features['ABV_numeric'] = features_df['ABV_numeric'].median()
        print(f"Warning: Invalid ABV value. Using median ({input_features['ABV_numeric']:.1f}) instead.")

    # Process geographic information (region and country)
    if region_name:
        # Look for matching region in the dataset
        matching_regions = features_df[features_df['RegionName'] == region_name]
        if not matching_regions.empty:
            # Use the latitude/longitude of the matching region
            input_features['latitude'] = matching_regions['latitude'].iloc[0]
            input_features['longitude'] = matching_regions['longitude'].iloc[0]
        else:
            print(f"Warning: Region '{region_name}' not found in dataset. Using default coordinates.")
            input_features['latitude'] = features_df['latitude'].median()
            input_features['longitude'] = features_df['longitude'].median()
    else:
        # If no region specified, use median values
        input_features['latitude'] = features_df['latitude'].median()
        input_features['longitude'] = features_df['longitude'].median()

    # Set default values for remaining features
    input_features['avg_rating'] = features_df['avg_rating'].median()
    input_features['rating_count'] = 0
    input_features['rating_std'] = 0

    # Create feature DataFrame and get recommendations
    input_df = pd.DataFrame([{col: input_features.get(col, 0) for col in feature_columns}])

    # Ensure all required columns are present
    missing_cols = [col for col in feature_columns if col not in input_df.columns]
    if missing_cols:
        print(f"Warning: Missing columns in input: {missing_cols}")
        for col in missing_cols:
            input_df[col] = 0

    # Get extra recommendations to account for filtering
    n_neighbors = min(100, len(features_df))  # Get up to 100 neighbors, but not more than dataset size

    try:
        # Scale the input features
        scaled_input = scaler.transform(input_df)

        # Get recommendations from the model
        distances, indices = knn_model.kneighbors(scaled_input, n_neighbors=n_neighbors)

        # Retrieve wine IDs
        recommended_indices = indices[0]
        recommended_wine_ids = features_df.iloc[recommended_indices]['WineID'].tolist()

        # Get full wine details - NOW INCLUDING HARMONIZE FIELD
        recommended_wines = features_df[features_df['WineID'].isin(recommended_wine_ids)][
            ['WineID', 'WineName', 'Country', 'Type', 'RegionName', 'Body', 'Acidity', 'Grapes_list', 'avg_rating', 'Harmonize']
        ]

        # Add similarity scores
        distance_map = {wine_id: dist for wine_id, dist in zip(recommended_wine_ids, distances[0])}
        recommended_wines['Similarity'] = recommended_wines['WineID'].map(lambda wine_id: 1 - distance_map[wine_id])

        # Process Harmonize field if it's stored as a string representation of a list
        import ast
        def safe_eval_list(x):
            """Safely convert string representations of lists to actual lists"""
            if isinstance(x, str):
                try:
                    return ast.literal_eval(x)
                except:
                    return []
            return x if isinstance(x, list) else []

        if 'Harmonize' in recommended_wines.columns:
            recommended_wines['Harmonize'] = recommended_wines['Harmonize'].apply(safe_eval_list)

        # Filter by country if specified
        if country:
            country_matches = recommended_wines[recommended_wines['Country'] == country]
            if not country_matches.empty:
                recommended_wines = country_matches
            else:
                print(f"Warning: No wines found from '{country}'. Showing wines from all countries.")

        # Sort by similarity and limit results
        result = recommended_wines.sort_values('Similarity', ascending=False).head(n_recommendations)

        if result.empty:
            print("No matching wines found. Try different criteria.")
            return pd.DataFrame()

        return result

    except Exception as e:
        print(f"Error getting recommendations: {str(e)}")
        return pd.DataFrame()

# Example usage
recommendations = get_wine_recommendations_by_characteristics(
    wine_type='Red',                   # Wine type (Red, White, Rosé, Sparkling, Dessert, Dessert/Port etc.)
    grape_varieties=['Corvina','Merlot'],       # Grape varieties (single string or list)
    body='Full-bodied',              # Body types: Very light-bodied, Light-bodied, Medium-bodied, Full-bodied, Very full-bodied
    abv=14.5,                          # Alcohol by volume percentage, min 0, max 50
    acidity=None,                      # Optional - Acidity (Low, Medium, High)
    country=None,                      # Optional - Country of origin (removed Italy to get more recommendations)
    region_name=None,                  # Optional - Region name
    n_recommendations=9                # Number of recommendations to return
)

# Display recommendations with food pairings
if not recommendations.empty:
    print("Recommended wines based on your preferences:")
    for i, (_, wine) in enumerate(recommendations.iterrows(), 1):
        grapes_str = ', '.join(wine['Grapes_list']) if isinstance(wine['Grapes_list'], list) else 'Unknown'

        # Format food pairings (from Harmonize field)
        food_pairings = "None specified"
        if 'Harmonize' in wine and isinstance(wine['Harmonize'], list) and wine['Harmonize']:
            food_pairings = ', '.join(wine['Harmonize'])

        print(f"{i}. {wine['WineName']} ({wine['Country']}, {wine['Type']})")
        print(f"   Region: {wine['RegionName'] or 'Unknown'}")
        print(f"   Grapes: {grapes_str}")
        print(f"   Body: {wine['Body'] or 'Unknown'}, Acidity: {wine['Acidity'] or 'Unknown'}")
        print(f"   Rating: {wine['avg_rating']:.1f}/5")
        print(f"   Food Pairings: {food_pairings}")  # Added food pairings display
        print(f"   Similarity: {wine['Similarity']:.2f}")
        print()
else:
    print("No recommendations found. Try adjusting your preferences.")

import ast
from collections import Counter

# Extract food pairings from all recommendations
all_food_pairings = []
for _, wine in recommendations.iterrows():
    if isinstance(wine['Harmonize'], list):
        all_food_pairings.extend(wine['Harmonize'])
    elif isinstance(wine['Harmonize'], str):
        # Handle case where Harmonize might be a string
        try:
            food_list = ast.literal_eval(wine['Harmonize'])
            if isinstance(food_list, list):
                all_food_pairings.extend(food_list)
        except:
            pass

# Count occurrences of each food pairing
food_counts = Counter(all_food_pairings)

# Find the most common food pairing
if food_counts:
    most_common_food = food_counts.most_common(1)[0][0]
    print(f"Our recommendation is that you pair your wine with {most_common_food}")
else:
    print("No food pairing recommendations available")

Recommended wines based on your preferences:
1. Prima Pietra Appassimento Leggero Rosso (Italy, Red)
   Region: Veneto
   Grapes: Merlot, Corvina
   Body: Full-bodied, Acidity: High
   Rating: 3.9/5
   Food Pairings: Beef, Pasta, Lamb, Game Meat
   Similarity: 0.98

2. Mille 1 Garda Rosso (Italy, Red)
   Region: Garda
   Grapes: Rebo, Merlot, Corvina
   Body: Full-bodied, Acidity: High
   Rating: 3.9/5
   Food Pairings: Beef, Pasta, Lamb, Game Meat
   Similarity: 0.97

3. Rosso Verona (Italy, Red)
   Region: Rosso Veronese
   Grapes: Corvina, Merlot
   Body: Full-bodied, Acidity: High
   Rating: 3.7/5
   Food Pairings: Beef, Pasta, Lamb, Game Meat
   Similarity: 0.97

4. Cormì Corvina-Merlot (Italy, Red)
   Region: Veneto
   Grapes: Corvina, Merlot
   Body: Full-bodied, Acidity: High
   Rating: 3.8/5
   Food Pairings: Beef, Pasta, Lamb, Game Meat
   Similarity: 0.97

5. Rosso Veneto (Italy, Red)
   Region: Veneto
   Grapes: Merlot, Corvina
   Body: Full-bodied, Acidity: High
   Rating:

/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mjh/.pyenv/versions/3.10.6/envs/cvino/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret

In [57]:
recommended_indices

NameError: name 'recommended_indices' is not defined

In [25]:
import ast
from collections import Counter

# Extract food pairings from all recommendations
all_food_pairings = []
for _, wine in recommendations.iterrows():
    if isinstance(wine['Harmonize'], list):
        all_food_pairings.extend(wine['Harmonize'])
    elif isinstance(wine['Harmonize'], str):
        # Handle case where Harmonize might be a string
        try:
            food_list = ast.literal_eval(wine['Harmonize'])
            if isinstance(food_list, list):
                all_food_pairings.extend(food_list)
        except:
            pass

# Count occurrences of each food pairing
food_counts = Counter(all_food_pairings)

# Find the most common food pairing
if food_counts:
    most_common_food = food_counts.most_common(1)[0][0]
    print(f"Our recommendation is that you pair your wine with {most_common_food}")
else:
    print("No food pairing recommendations available")

Our recommendation is that you pair your wine with Beef


In [26]:
# WINE RECOMMENDATION FUNCTION BY FOOD PAIRING
import pandas as pd
import numpy as np
import ast
from collections import Counter

def get_wine_recommendations_by_food(
    food_pairing,                    # Required: Food you want to pair with (e.g., "steak", "pasta")
    wine_type=None,                  # Optional: Wine type (Red, White, Rosé, Sparkling, etc.)
    grape_varieties=None,            # Optional: Grape varieties (single string or list)
    body=None,                       # Optional: Body (Very light to Very full-bodied)
    abv=None,                        # Optional: Alcohol by volume percentage
    acidity=None,                    # Optional: Acidity (Low, Medium, High)
    country=None,                    # Optional: Country of origin
    region_name=None,                # Optional: Region name
    n_recommendations=5,             # Number of recommendations to return
    exact_match_only=False           # If True, only return wines with exact food match
):
    """
    Get wine recommendations based on food pairing and optional wine characteristics.

    This function performs a reverse lookup - instead of starting with wine
    characteristics and finding food pairings, it starts with the desired food
    and finds wines that pair well with it.

    Parameters:
    -----------
    food_pairing : str
        The food you want to pair wine with (e.g., "steak", "pasta", "chicken")
    wine_type : str, optional
        Type of wine (e.g., "Red", "White", "Rosé", "Sparkling")
    grape_varieties : str or list, optional
        Specific grape varieties you prefer
    body : str, optional
        Wine body preference (e.g., "Light-bodied", "Medium-bodied", "Full-bodied")
    abv : float, optional
        Desired alcohol by volume percentage
    acidity : str, optional
        Desired acidity level ("Low", "Medium", "High")
    country : str, optional
        Country of origin preference
    region_name : str, optional
        Wine region preference
    n_recommendations : int, default=5
        Number of wine recommendations to return
    exact_match_only : bool, default=False
        If True, only return wines that explicitly list the food_pairing

    Returns:
    --------
    pd.DataFrame
        DataFrame containing recommended wines with their details
    """
    # Step 1: Prepare food pairing search term
    # Convert to lowercase for case-insensitive matching
    food_search = food_pairing.lower().strip()

    # Step 2: Find wines that pair with this food
    # First, ensure Harmonize is properly processed as a list
    def safe_eval_list(x):
        """Safely convert string representations of lists to actual lists"""
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except:
                return []
        return x if isinstance(x, list) else []

    # Create a working copy of the features dataframe with processed Harmonize field
    working_df = features_df.copy()
    working_df['Harmonize'] = working_df['Harmonize'].apply(safe_eval_list)

    # Function to check if a food is mentioned in the harmonize list
    def contains_food(harmonize_list, food_term):
        """Check if any item in the harmonize list contains the food term"""
        if not isinstance(harmonize_list, list):
            return False

        # Convert all harmonize items to lowercase for case-insensitive matching
        harmonize_lower = [item.lower() for item in harmonize_list]

        # Check for exact matches first
        if food_term in harmonize_lower:
            return True

        # If not exact, check if food term is part of any harmonize item
        if not exact_match_only:
            for item in harmonize_lower:
                if food_term in item:
                    return True

        return False

    # Create a mask for wines that pair with this food
    working_df['food_match'] = working_df['Harmonize'].apply(
        lambda x: contains_food(x, food_search))

    # Step 3: Initial filtering - get wines that match the food pairing
    food_matched_wines = working_df[working_df['food_match']]

    # Check if any wines match the food pairing
    if food_matched_wines.empty:
        print(f"No wines found that pair with '{food_pairing}'. Try a different food.")
        return pd.DataFrame()

    # Step 4: Apply additional filters based on optional parameters
    if wine_type is not None:
        food_matched_wines = food_matched_wines[
            food_matched_wines['Type'] == wine_type]

    if grape_varieties is not None:
        # Convert to list if it's a string
        if isinstance(grape_varieties, str):
            if ',' in grape_varieties:
                grape_list = [g.strip() for g in grape_varieties.split(',')]
            else:
                grape_list = [grape_varieties]
        else:
            grape_list = grape_varieties

        # Filter for wines that contain any of these grapes
        def has_any_grape(wine_grapes, target_grapes):
            if not isinstance(wine_grapes, list):
                return False
            return any(grape in wine_grapes for grape in target_grapes)

        food_matched_wines = food_matched_wines[
            food_matched_wines['Grapes_list'].apply(
                lambda x: has_any_grape(x, grape_list))]

    if body is not None:
        food_matched_wines = food_matched_wines[
            food_matched_wines['Body'] == body]

    if country is not None:
        food_matched_wines = food_matched_wines[
            food_matched_wines['Country'] == country]

    if acidity is not None:
        food_matched_wines = food_matched_wines[
            food_matched_wines['Acidity'] == acidity]

    if region_name is not None:
        food_matched_wines = food_matched_wines[
            food_matched_wines['RegionName'] == region_name]

    # If after all filters we have no wines left, return empty result
    if food_matched_wines.empty:
        print(f"No wines found that match all your criteria with '{food_pairing}'.")
        return pd.DataFrame()

    # Sort by rating and return top matches
    result = food_matched_wines.sort_values('avg_rating', ascending=False).head(n_recommendations)
    return result

# Example usage
food_recommendations = get_wine_recommendations_by_food(
    food_pairing="Beef",         # Food you want to pair with
    wine_type='Red',                   # Optional: Type of wine
    grape_varieties=None,              # Optional: Specific grape varieties
    body=None,                         # Body types: Very light-bodied, Light-bodied, Medium-bodied, Full-bodied, Very full-bodied
    acidity=None,                      # Optional: Acidity level
    country='Argentina',                      # Optional: Country preference
    n_recommendations=3                # Number of recommendations
)

# Display recommendations
if not food_recommendations.empty:
    print(f"Recommended wines to pair with your food:")
    for i, (_, wine) in enumerate(food_recommendations.iterrows(), 1):
        grapes_str = ', '.join(wine['Grapes_list']) if isinstance(wine['Grapes_list'], list) else 'Unknown'

        # Format food pairings (from Harmonize field)
        food_pairings = "None specified"
        if isinstance(wine['Harmonize'], list) and wine['Harmonize']:
            food_pairings = ', '.join(wine['Harmonize'])

        # Highlight if this wine explicitly matches the food
        food_match_indicator = " ✓" if wine.get('food_match', False) else ""

        print(f"{i}. {wine['WineName']} ({wine['Country']}, {wine['Type']}){food_match_indicator}")
        print(f"   Region: {wine['RegionName'] or 'Unknown'}")
        print(f"   Grapes: {grapes_str}")
        print(f"   Body: {wine['Body'] or 'Unknown'}, Acidity: {wine['Acidity'] or 'Unknown'}")
        print(f"   Rating: {wine['avg_rating']:.1f}/5")
        print(f"   Food Pairings: {food_pairings}")
        print()
else:
    print("No wine recommendations found for your food. Try a different food or criteria.")

Recommended wines to pair with your food:
1. Cobos Volturno (Argentina, Red) ✓
   Region: Mendoza
   Grapes: Cabernet Sauvignon
   Body: Full-bodied, Acidity: High
   Rating: 4.8/5
   Food Pairings: Beef, Lamb, Game Meat, Poultry

2. Cobos Chañares Estate Malbec (Argentina, Red) ✓
   Region: Mendoza
   Grapes: Malbec
   Body: Very full-bodied, Acidity: High
   Rating: 4.8/5
   Food Pairings: Beef, Lamb, Poultry

3. Cobos Corte uNico (Argentina, Red) ✓
   Region: Mendoza
   Grapes: Cabernet Sauvignon, Malbec
   Body: Full-bodied, Acidity: High
   Rating: 4.8/5
   Food Pairings: Beef, Lamb, Game Meat, Poultry



In [27]:
import ast
from collections import Counter

def extract_unique_harmonize_values(dataframe):
    """
    Extracts all unique values from the Harmonize column.

    Args:
        dataframe: The DataFrame containing the Harmonize column

    Returns:
        A tuple containing:
        - A sorted list of unique food pairing values
        - A list of all food pairings (including duplicates)
    """
    # Helper function to safely convert string representations of lists to actual lists
    def safe_eval_list(x):
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except:
                return []
        return x if isinstance(x, list) else []

    # Process the Harmonize column to ensure all values are proper lists
    processed_harmonize = dataframe['Harmonize'].apply(safe_eval_list)

    # Flatten the list of lists into a single list of all food pairings
    all_pairings = []
    for pairings_list in processed_harmonize:
        if isinstance(pairings_list, list):
            all_pairings.extend(pairings_list)

    # Get unique values
    unique_pairings = sorted(set(all_pairings))

    return unique_pairings, all_pairings

# Extract unique values and all pairings from the Harmonize column
unique_food_pairings, all_food_pairings = extract_unique_harmonize_values(features_df)

# Print the result
print(f"Found {len(unique_food_pairings)} unique food pairings:")
for i, pairing in enumerate(unique_food_pairings, 1):
    print(f"{i}. {pairing}")

# Get counts of each food pairing
pairing_counts = Counter(all_food_pairings)

# Print the top 20 most common pairings
print("\nTop 20 most common food pairings:")
for pairing, count in pairing_counts.most_common(20):
    print(f"{pairing}: {count} wines")

Found 66 unique food pairings:
1. Aperitif
2. Appetizer
3. Asian Food
4. Baked Potato
5. Barbecue
6. Beans
7. Beef
8. Blue Cheese
9. Cake
10. Cheese
11. Chestnut
12. Chicken
13. Chocolate
14. Citric Dessert
15. Codfish
16. Cold Cuts
17. Cookies
18. Cream
19. Cured Meat
20. Curry Chicken
21. Dessert
22. Dried Fruits
23. Duck
24. Eggplant Parmigiana
25. Fish
26. French Fries
27. Fruit
28. Fruit Dessert
29. Game Meat
30. Goat Cheese
31. Grilled
32. Ham
33. Hard Cheese
34. Lamb
35. Lasagna
36. Lean Fish
37. Light Stews
38. Maturated Cheese
39. Meat
40. Medium-cured Cheese
41. Mild Cheese
42. Mushrooms
43. Paella
44. Pasta
45. Pizza
46. Pork
47. Poultry
48. Rich Fish
49. Risotto
50. Roast
51. Salad
52. Sashimi
53. Seafood
54. Shellfish
55. Snack
56. Soft Cheese
57. Soufflé
58. Spiced Fruit Cake
59. Spicy Food
60. Sushi
61. Sweet Dessert
62. Tagliatelle
63. Tomato Dishes
64. Veal
65. Vegetarian
66. Yakissoba

Top 20 most common food pairings:
Poultry: 58315 wines
Beef: 57774 wines
Lamb: 3872

In [47]:
wine_type='Red',                # Wine type (Red, White, Rosé, Sparkling, etc.)
grape_varieties='Syrah/Shiraz',       # Grape varieties (single string or list)
body='Full-bodied',           # Body ['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']
acidity='Medium',               # Acidity (Low, Medium, High)
abv=14.0,                       # Alcohol by volume percentage
country='Australia',               # Country of origin
region_name='MacLaren Vale',  # Region name
n_recommendations=5             # Number of recommendations to return


    # create a DataFrame

X_pred = pd.DataFrame(dict(
    Type =[wine_type],
    ABV=[abv],
    Body=body,
    Acidity=[acidity],
    Country=[country],
    RegionName=[region_name],
    latitude = [41.8719],
    longitude = [12.5674],
    Grapes_list = grape_varieties,
    ))

In [48]:
X_pred

,Type,ABV,Body,Acidity,Country,RegionName,latitude,longitude,Grapes_list
0,"(Red,)","(14.0,)",Full-bodied,"(Medium,)","(Australia,)","(MacLaren Vale,)",41.8719,12.5674,Syrah/Shiraz
